In [1]:
import os
import pyspark
conf = pyspark.SparkConf()
conf.set("spark.driver.memory", "8g")
conf.set("spark.worker.timeout", "10000000")
conf.set("spark.driver.maxResultSize", "0")
conf.set("spark.executor.memory", "8g")
conf.set("spark.driver.extraClassPath", "/usr/local/cuda-11.2/lib64") \
                 .set("spark.executor.extraClassPath", "/usr/local/cuda-11.2/lib64") \
                 .set("spark.driver.extraLibraryPath", "/usr/local/cuda-11.2/lib64") \
                 .set("spark.executor.extraLibraryPath", "/usr/local/cuda-11.2/lib64") \
                 .set("spark.rapids.sql.concurrentGpuTasks", "") \
                 .set("spark.rapids.sql.variableFloatAgg.enabled", "true")
spark = pyspark.SparkContext(conf=conf)
spark


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/09 14:22:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/09 14:22:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/05/09 14:22:34 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


<SparkContext master=local[*] appName=pyspark-shell>

In [2]:
conf.set('spark.sql.repl.eagerEval.enabled', True)

spark = pyspark.SQLContext.getOrCreate(spark)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [3]:
df = spark.read.option("multiline",True)\
              .option("mode", "DROPMALFORMED")\
              .option("dateFormat", "yyyy-MM-dd")\
              .option("timestampFormat", "yyyy-MM-dd HH:mm:ss")\
              .csv("/Users/sejalrameshjagtap/Documents/bigdata/filtered_news.csv", header=True)


In [ ]:
from pyspark.sql.functions import isnull, when, count, col

df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

In [ ]:
df = df.drop("meta_description")

In [4]:
df_million = df.limit(100000)

In [5]:
df_million = df_million.select(['type', 'content'])

In [6]:
df_million.show()

+----------+--------------------+
|      type|             content|
+----------+--------------------+
|     rumor|Life is an illusi...|
|      hate|Unfortunately, he...|
|      hate|The Los Angeles P...|
|      hate|The White House h...|
|      hate|“The time has com...|
|      hate|The Central Ameri...|
|unreliable|UN-Backed Police ...|
|unreliable|It should have co...|
|unreliable|“When the police ...|
|unreliable|Zambia Must Clari...|
|conspiracy|"""…I have set be...|
|conspiracy|"Why We Oppose Pl...|
| clickbait|The website from ...|
|    satire|For as long as he...|
|    satire|For as long as he...|
|unreliable|Most people’s pet...|
|unreliable|Dogs are fantasti...|
|unreliable|The lion may be k...|
|unreliable|Stories of animal...|
| clickbait|Sarah Huckabee Sa...|
+----------+--------------------+
only showing top 20 rows



In [7]:
df_million = df_million.dropna()

In [8]:
df_million.count()

99997

In [9]:
from pyspark.sql.functions import concat_ws
from pyspark.ml.feature import Tokenizer, CountVectorizer, StringIndexer
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Load data into DataFrame

# Concatenate text columns
# df_thousand = df_thousand.withColumn('text', concat_ws(' ', 'title', 'content'))

# Tokenize text data
tokenizer = Tokenizer(inputCol='content', outputCol='words')
tokenized = tokenizer.transform(df_million)

# Convert words to numerical feature vectors
cv = CountVectorizer(inputCol='words', outputCol='features')
cvmodel = cv.fit(tokenized)

vectorized =cvmodel.transform(tokenized)
# Convert label column to numerical index
label_indexer = StringIndexer(inputCol='type', outputCol='label_index')
indexed = label_indexer.fit(vectorized).transform(vectorized)

ERROR:root:KeyboardInterrupt while sending command.                 (0 + 1) / 1]
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [16]:
vectorized.write.format('pickle').save('vectorized.pkl')


ConnectionRefusedError: [Errno 61] Connection refused

In [12]:
cvmodel.save("my_vectorizer_model_big")

23/05/09 03:55:46 WARN TaskSetManager: Stage 26 contains a task of very large size (5164 KiB). The maximum recommended task size is 1000 KiB.


In [13]:
indexed = indexed.select('content', 'features', 'label_index')
indexed

DataFrame[content: string, features: vector, label_index: double]

In [14]:
# Split data into training and testing sets
train, test = indexed.randomSplit([0.8, 0.2], seed=1234)

In [15]:
# Instantiate and train model
rf = RandomForestClassifier(labelCol='label_index', featuresCol='features',maxDepth=30, numTrees=20)
model = rf.fit(train)

# Make predictions on testing data
predictions = model.transform(test)

# Evaluate performance
evaluator = MulticlassClassificationEvaluator(labelCol='label_index', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f'Accuracy = {accuracy:.2f}')


23/05/09 03:56:49 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
23/05/09 03:57:22 WARN DAGScheduler: Broadcasting large task binary with size 3.0 MiB
23/05/09 03:57:47 WARN DAGScheduler: Broadcasting large task binary with size 5.6 MiB
23/05/09 04:02:27 WARN DAGScheduler: Broadcasting large task binary with size 1033.7 KiB
23/05/09 04:02:29 WARN DAGScheduler: Broadcasting large task binary with size 8.7 MiB
23/05/09 04:03:10 WARN MemoryStore: Not enough space to cache rdd_94_0 in memory! (computed 3.3 GiB so far)
23/05/09 04:03:10 WARN BlockManager: Persisting block rdd_94_0 to disk instead.
23/05/09 05:11:10 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1039557 ms exceeds timeout 120000 ms
23/05/09 05:11:10 WARN SparkContext: Killing executors is not supported by current scheduler.
23/05/09 05:11:13 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org

KeyboardInterrupt: 

In [ ]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label_index', metricName = 'accuracy')
print('Random Forest classifier Accuracy:', multi_evaluator.evaluate(predictions))

Random Forest classifier Accuracy: 0.4069767441860465


In [ ]:
model.save("RFC_Model_big.mdl")

TypeError: MLWritable.save() takes 2 positional arguments but 3 were given

In [ ]:
# model.save(spark, "RFC_Model_1.mdl")

In [ ]:
# Load new data into DataFrame
new_data = spark.read.option("multiline",True)\
              .option("mode", "DROPMALFORMED")\
              .option("dateFormat", "yyyy-MM-dd")\
              .option("timestampFormat", "yyyy-MM-dd HH:mm:ss")\
              .csv("/content/drive/MyDrive/Fake News Dataset/news_sample.csv", header=True)

In [ ]:
new_data = new_data.select('content', 'title')

In [ ]:
new_data = new_data.dropna().limit(10)
new_data.show()

+--------------------+--------------------+
|             content|               title|
+--------------------+--------------------+
|Sometimes the pow...|Church Congregati...|
|AWAKENING OF 12 S...|AWAKENING OF 12 S...|
|Never Hike Alone:...|Never Hike Alone ...|
|When a rare shark...|Elusive ‘Alien Of...|
|Donald Trump has ...|Trump’s Genius Po...|
|“Republicans and ...| Black Agenda Report|
|Could you imagine...|waking up in the ...|
|Citizen Journalis...|  Citizen Journalist|
|Usa Dollar Tanks ...|Usa Dollar Tanks ...|
|Subscribe to Cana...|It’s Not Really P...|
+--------------------+--------------------+



In [ ]:
# Concatenate text columns
new_data = new_data.withColumn('text', concat_ws(' ', 'title', 'content'))

# Tokenize text data
tokenizer = Tokenizer(inputCol='text', outputCol='words')
tokenized = tokenizer.transform(new_data)

# Convert words to numerical feature vectors
vectorizer = CountVectorizer(inputCol='words', outputCol='features')
vectorized = vectorizer.fit(tokenized).transform(tokenized)



In [ ]:
vectorized = vectorized.select('title', 'content', 'features')
vectorized

title,content,features
Is life an ILLUSI...,Life is an illusi...,"(46922,[0,1,2,3,4..."
Donald Trump,"Unfortunately, he...","(46922,[0,3,6,7,9..."
Donald Trump,The Los Angeles P...,"(46922,[0,1,3,6,7..."
MORE WINNING! Isr...,The White House h...,"(46922,[0,1,2,3,4..."
"“Oh, Trump, you c...",“The time has com...,"(46922,[0,1,2,3,6..."
Following Guatema...,The Central Ameri...,"(46922,[0,1,2,3,4..."
UN-Backed Police ...,UN-Backed Police ...,"(46922,[0,1,2,3,4..."
Black Agenda Report,It should have co...,"(46922,[0,6,7,11,..."
Black Agenda Report,“When the police ...,"(46922,[0,2,6,48,..."
Zambia Must Clari...,Zambia Must Clari...,"(46922,[0,1,2,3,4..."


In [ ]:
# Make predictions using trained model
predictions = model.transform(vectorized)

# Show predicted labels and probabilities
predictions.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|               title|             content|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|Is life an ILLUSI...|Life is an illusi...|(46922,[0,1,2,3,4...|[4.34172761673744...|[0.21708638083687...|       0.0|
|        Donald Trump|Unfortunately, he...|(46922,[0,3,6,7,9...|[4.61707246731230...|[0.23085362336561...|       0.0|
|        Donald Trump|The Los Angeles P...|(46922,[0,1,3,6,7...|[4.61707246731230...|[0.23085362336561...|       0.0|
|MORE WINNING! Isr...|The White House h...|(46922,[0,1,2,3,4...|[3.75989083143385...|[0.18799454157169...|       2.0|
|“Oh, Trump, you c...|“The time has com...|(46922,[0,1,2,3,6...|[4.29791337986223...|[0.21489566899311...|       0.0|
|Following Guatema...|The Central Ameri...|(46922,[0,1,2